In [1]:
# ====================================================
# CFG
# ====================================================
class CFG:
    wandb=True
    competition='FB3'
    _wandb_kernel='nakama'
    debug=False
    apex=True
    print_freq=20
    num_workers=4
    model="microsoft/deberta-v3-base"
    gradient_checkpointing=True
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=8
    epochs=3
    encoder_lr=5e-5
    decoder_lr=1e-4
    min_lr=1e-7
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=1
    max_len=None
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    train=True
    llrd = 0.97
    loss_weight = [0.21,0.16,0.10, 0.16, 0.21,0.16]
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [2]:
OUTPUT_DIR = '/home/u210810417/'

In [3]:
# wandb
# ====================================================
if CFG.wandb:
    
    import wandb

    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        secret_value_0 = user_secrets.get_secret("wandb_api")
        wandb.login(key=secret_value_0)
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project='FB3-Public', 
                     name=CFG.model,
                     config=class2dict(CFG),
                     group=CFG.model,
                     job_type="train",
                     anonymous=anony)

If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. 
Get your W&B access token from here: https://wandb.ai/authorize


wandb: Currently logged in as: anony-mouse-430222. Use `wandb login --relogin` to force relogin


In [4]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

os.system('pip install iterative-stratification==0.1.7')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

# os.system('pip uninstall -y transformers')
# os.system('pip uninstall -y tokenizers')
# os.system('python -m pip install --no-index --find-links=../input/fb3-pip-wheels transformers')
# os.system('python -m pip install --no-index --find-links=../input/fb3-pip-wheels tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Defaulting to user installation because normal site-packages is not writeable
tokenizers.__version__: 0.12.1
transformers.__version__: 4.21.2
env: TOKENIZERS_PARALLELISM=true


In [5]:
# ====================================================
# Utils
# ====================================================
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [6]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv('/home/u210810417/train.csv')
test = pd.read_csv('/home/u210810417/test.csv')
submission = pd.read_csv('/home/u210810417/sample_submission.csv')

print(f"train.shape: {train.shape}")
display(train.head())
print(f"test.shape: {test.shape}")
display(test.head())
print(f"submission.shape: {submission.shape}")
display(submission.head())

train.shape: (3911, 8)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


test.shape: (3, 2)


,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


submission.shape: (3, 7)


,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


In [7]:
# ====================================================
# CV split
# ====================================================
Fold = MultilabelStratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train[CFG.target_cols])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

fold
0    978
1    977
2    978
3    978
dtype: int64

In [8]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
tokenizer.save_pretrained(OUTPUT_DIR)
CFG.tokenizer = tokenizer

wandb: Network error (TransientError), entering retry loop.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# ====================================================
# Define max_len
# ====================================================
lengths = []
tk0 = tqdm(train['full_text'].fillna("").values, total=len(train))
for text in tk0:
    length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
    lengths.append(length)
CFG.max_len = max(lengths) + 3 # cls & sep & sep
# CFG.max_len = 512
LOGGER.info(f"max_len: {CFG.max_len}")

  0%|          | 0/3911 [00:00<?, ?it/s]

max_len: 1429


In [10]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        max_length=CFG.max_len,
        pad_to_max_length=True,
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['full_text'].values
        self.labels = df[cfg.target_cols].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        label = torch.tensor(self.labels[item], dtype=torch.float)
        return inputs, label
    

def collate(inputs):
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs

In [11]:
# ====================================================
# Model
# ====================================================
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings
    
    
# class WeightedLayerPooling(nn.Module):
#     def __init__(self, num_hidden_layers, layer_start: int = 9, layer_weights = None):
#         super(WeightedLayerPooling, self).__init__()
#         self.layer_start = layer_start
#         self.num_hidden_layers = num_hidden_layers
#         self.layer_weights = layer_weights if layer_weights is not None \
#             else nn.Parameter(
#                 torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
#             )

#     def forward(self, features):

#         all_layer_embedding = torch.stack(features)
#         all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]

#         weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
#         weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

#         return weighted_average
    

class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
            self.config.output_hidden_layers = True
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
            self.config.output_hidden_layers = True
        else:
            self.model = AutoModel(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
#         self.config.update({'output_hidden_states':True})
        # self.pool = WeightedLayerPooling(self.config.num_hidden_layers)
        self.pool = MeanPooling()
        
        in_dim = self.config.hidden_size
        self.attention = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.LayerNorm(in_dim),
            nn.GELU(),
            nn.Linear(in_dim, 1),
            )
        
        self.fc = nn.Linear(self.config.hidden_size, 6)

        self._init_weights(self.fc)
        
        
        
        for module in self.attention.modules():
            self._init_weights(module)
        
        
        
        
        
        
        REINIT_LAYERS = 1
        for layer in self.model.encoder.layer[-REINIT_LAYERS:]:
            for module in layer.modules():
                self._init_weights(module)
            print('Done')
     
            
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight = torch.nn.init.kaiming_normal_(module.weight)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
            

    
    

##############################################
#     def forward(self, inputs):
# #         self.config.update({'output_hidden_states':True})
#         outputs = self.model(**inputs)
        
        
#         feature = self.pool(outputs.hidden_states)

#         return self.fc(feature)


#######################################################
#     def feature(self, inputs):
#         outputs = self.model(**inputs)
#         last_hidden_states = outputs[0]
#         feature = self.pool(last_hidden_states, inputs['attention_mask'])
#         return feature

#     def forward(self, inputs):
#         feature = self.feature(inputs)
#         output = self.fc(feature)
#         return output

##########################################################
    def forward(self,inputs):

        outputs = self.model(**inputs)
        x = outputs[0]
        mask = inputs['attention_mask']
        w = self.attention(x).float()
        w[mask==0]=float('-inf')
        w = torch.softmax(w,1)
        x = torch.sum(w * x, dim=1)
        return self.fc(x)


In [12]:
# ====================================================
# Loss
# ====================================================
class RMSELoss(nn.Module):
    def __init__(self, reduction='mean', eps=1e-9):
        super().__init__()
        self.mse = nn.MSELoss(reduction='none')
        self.reduction = reduction
        self.eps = eps

    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        if self.reduction == 'none':
            loss = loss
        elif self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
        return loss

In [13]:
class Weighted_Loss(nn.Module):
    def __init__(self,criterion):
        super().__init__()
        self.criterion = criterion
    def forward(self,y_pred,y_true):
        loss = 0
        for i in range(len(CFG.target_cols)):
            loss += self.criterion(y_pred[:,i],y_true[:,i])*CFG.loss_weight[i]
        return loss
        

In [14]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
            loss = criterion(y_preds, labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs)
            loss = criterion(y_preds, labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [15]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_cols].values
    
    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.9):
        
        
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        layer_names = []

        for idx, (name, param) in enumerate(model.model.named_parameters()):
            if not any(nd in name for nd in no_decay):
                layer_names.append(name)
            # print(f'{idx}: {name}')
            
        layer_names.reverse()
        lr      = encoder_lr
        lr_mult = CFG.llrd
        #0.995 0.4444
        # print(layer_names)
        # placeholder
        parameters = []
        
        
        
        # store params & learning rates
        for idx, name in enumerate(layer_names):
    # display info
            # print(f'{idx}: lr = {lr:.6f}, {name}')
    
    # append layer parameters
            parameters += [{'params': [p for n, p in model.model.named_parameters() if n == name],
                    'lr':lr, 'weight_decay':weight_decay}]
    
    # update learning rate
            lr *= lr_mult
        parameters +=[{'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}]
        for n,p in model.named_parameters():
            if "model" not in n:
                if "attention" in n:
                    print(n)
        # print(parameters)
        # print(parameters)
        return parameters

#     def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.9):
        
        
#         no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
#         layer_names = []

#         for idx, (name, param) in enumerate(model.model.named_parameters()):
#             if not any(nd in name for nd in no_decay):
#                 layer_names.append(name)
#             # print(f'{idx}: {name}')
            
#         layer_names.reverse()
#         lr      = encoder_lr
#         lr_mult = 0.993
#         #0.995 0.4444
#         # print(layer_names)
#         # placeholder
#         parameters = []
        
        
        
#         # store params & learning rates
#         for idx, name in enumerate(layer_names):
#     # display info
#             # print(f'{idx}: lr = {lr:.6f}, {name}')
    
#     # append layer parameters
#             parameters += [{'params': [p for n, p in model.model.named_parameters() if n == name],
#                     'lr':lr, 'weight_decay':weight_decay}]
    
#     # update learning rate
#             lr *= lr_mult
#         parameters +=[{'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
#              'lr': encoder_lr, 'weight_decay': 0.0},
#             {'params': [p for n, p in model.named_parameters() if "model" not in n and "attention" in n],
#              'lr': encoder_lr, 'weight_decay': 0.0},
#             {'params': [p for n, p in model.named_parameters() if "model" not in n and "attention" not in n],
#              'lr': decoder_lr, 'weight_decay': 0.0}]
#         for n,p in model.named_parameters():
#             if "model" not in n:
#                 if "attention" in n:
#                     print(n)
#         # print(parameters)
#         # print(parameters)
#         return parameters
    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = Weighted_Loss(nn.SmoothL1Loss(reduction='mean')) # RMSELoss(reduction="mean")
 
    best_score = np.inf

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        
        # scoring
        score, scores = get_score(valid_labels, predictions)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}  Scores: {scores}')
        if CFG.wandb:
            wandb.log({f"[fold{fold}] epoch": epoch+1, 
                       f"[fold{fold}] avg_train_loss": avg_loss, 
                       f"[fold{fold}] avg_val_loss": avg_val_loss,
                       f"[fold{fold}] score": score})
        
        if best_score > score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")

    predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds[[f"pred_{c}" for c in CFG.target_cols]] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [16]:
# if __name__ == '__main__':
    
#     def get_result(oof_df):
#         labels = oof_df[CFG.target_cols].values
#         preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
#         score, scores = get_score(labels, preds)
#         LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')
    
#     if CFG.train:
#         oof_df = pd.DataFrame()
#         for fold in range(CFG.n_fold):
#             if fold in CFG.trn_fold:
#                 _oof_df = train_loop(train, fold)
#                 oof_df = pd.concat([oof_df, _oof_df])
#                 LOGGER.info(f"========== fold: {fold} result ==========")
#                 get_result(_oof_df)
#         oof_df = oof_df.reset_index(drop=True)
#         LOGGER.info(f"========== CV ==========")
#         get_result(oof_df)
#         oof_df.to_pickle(OUTPUT_DIR+'oof_df.pkl')
        
#     if CFG.wandb:
#         wandb.finish()

In [17]:
import optuna
    
def get_result(oof_df):
        labels = oof_df[CFG.target_cols].values
        preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
        score, scores = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')
        return score
def objective(trial):

    CFG.print_freq = np.inf
    # CFG.num_warmup_step = trial.suggest_int('num_warmup_step',
    CFG.epoch = trial.suggest_int('epoch',3,4)
    encoder_lr = trial.suggest_float('encoder_lr',5e-6,5e-4)
    decoder_lr = 2*encoder_lr
    llrd = trial.suggest_float('llrd',0.8,0.99)

    oof_df = pd.DataFrame()
    for fold in range(CFG.n_fold):
        if fold in CFG.trn_fold:
            _oof_df = train_loop(train, fold)
            oof_df = pd.concat([oof_df, _oof_df])
            # LOGGER.info(f"========== fold: {fold} result ==========")
            
    oof_df = oof_df.reset_index(drop=True)
    print(11111111111111111111111111111111)
    LOGGER.info(f"========== CV ==========")
    score = get_result(oof_df)
    print(score)
    return score
        
            
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, timeout=40000)

    pruned_trials = study.get_trials(deepcopy=False)
    complete_trials = study.get_trials(deepcopy=False)

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2022-12-14 01:48:52,156] A new study created in memory with name: no-name-7f767321-9887-4d3b-8957-db8e28813799
========== fold: 0 training ==========
DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-base",
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.21.2",
  "type_vo

Done
attention.0.weight
attention.0.bias
attention.1.weight
attention.1.bias
attention.3.weight
attention.3.bias
Epoch: [1][0/2933] Elapsed 0m 1s (remain 88m 37s) Loss: 2.1010(2.1010) Grad: inf  LR: 0.00000625  
Epoch: [1][2932/2933] Elapsed 6m 50s (remain 0m 0s) Loss: 0.0917(0.1762) Grad: 186656.6875  LR: 0.00003754  
EVAL: [0/489] Elapsed 0m 0s (remain 2m 44s) Loss: 0.0836(0.0836) 


Epoch 1 - avg_train_loss: 0.1762  avg_val_loss: 0.1180  time: 450s
Epoch 1 - Score: 0.4783  Scores: [0.5388780363028517, 0.4407438434289395, 0.41600740238790335, 0.4945145057247125, 0.500524024628813, 0.4792564375476278]
Epoch 1 - Save Best Score: 0.4783 Model


EVAL: [488/489] Elapsed 0m 39s (remain 0m 0s) Loss: 0.0891(0.1180) 
Epoch: [2][0/2933] Elapsed 0m 0s (remain 18m 42s) Loss: 0.0764(0.0764) Grad: 360640.2500  LR: 0.00003753  
Epoch: [2][2932/2933] Elapsed 6m 50s (remain 0m 0s) Loss: 0.4474(0.1145) Grad: 406937.9375  LR: 0.00001252  
EVAL: [0/489] Elapsed 0m 0s (remain 2m 35s) Loss: 0.0994(0.0994) 


Epoch 2 - avg_train_loss: 0.1145  avg_val_loss: 0.1094  time: 450s
Epoch 2 - Score: 0.4607  Scores: [0.48272728814153354, 0.44295246831876356, 0.4121399877199979, 0.4548005388909946, 0.5205580824786445, 0.4509041387020568]
Epoch 2 - Save Best Score: 0.4607 Model


...

Epoch 3 - avg_train_loss: 0.0918  avg_val_loss: 0.1004  time: 348s
Epoch 3 - Score: 0.4440  Scores: [0.4807670287807327, 0.4434742693413729, 0.40963217531780977, 0.43559801052061237, 0.464886314218015, 0.42945756491663684]
Epoch 3 - Save Best Score: 0.4440 Model


EVAL: [488/489] Elapsed 0m 25s (remain 0m 0s) Loss: 0.0546(0.1004) 


========== CV ==========
Score: 0.4516  Scores: [0.4828283467175599, 0.44407167999176844, 0.41490833398215304, 0.45247093741560873, 0.4732360743123661, 0.44201340710333087]
[I 2022-12-14 14:03:43,234] Trial 9 finished with value: 0.4515881299204645 and parameters: {'epoch': 4, 'encoder_lr': 0.000134831353541911, 'llrd': 0.9103563486857054}. Best is trial 5 with value: 0.45024024643677324.


11111111111111111111111111111111
0.4515881299204645


NameError: name 'TrialState' is not defined

In [18]:
pruned_trials = study.get_trials(deepcopy=False)
complete_trials = study.get_trials(deepcopy=False)

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  10
  Number of pruned trials:  10
  Number of complete trials:  10
Best trial:
  Value:  0.45024024643677324
  Params: 
    epoch: 3
    encoder_lr: 3.007918911639952e-05
    llrd: 0.8329610635006021
